<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem%20LLC%20Extraction%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

In [2]:
!wget -O MLNN.py "https://raw.githubusercontent.com/vladgap/MLNN/main/MLNN_3_2.py"
from MLNN import *

--2025-02-21 06:09:58--  https://raw.githubusercontent.com/vladgap/MLNN/main/MLNN_3_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14169 (14K) [text/plain]
Saving to: ‘MLNN.py’

MLNN.py             100%[===================>]  13.84K  --.-KB/s    in 0s      

2025-02-21 06:09:59 (53.2 MB/s) - ‘MLNN.py’ saved [14169/14169]

Version of MLNN is 3.2


In [3]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_2.py"
import LLC

--2025-02-21 06:09:59--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44371 (43K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  43.33K  --.-KB/s    in 0.007s  

2025-02-21 06:09:59 (6.14 MB/s) - ‘LLC.py’ saved [44371/44371]



In [4]:
# import LLC_5_2 as LLC52

In [5]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_extr1
data 8 -- EQUIL_rotem_extraction 2+3

In [6]:
# X: 'H3PO4_aq'	'SO4_aq'
# T: 'FA_aq'	'solvent_aq'	'H3PO4_org'	'SO4_org'	'water_org'

scaler_data=[[37.23870967741936, 7.199806451612903],
 [21.87534593155935, 6.51727589252697],
 [155.77096774193546,
  0.5283870967741936,
  13.459354838709675,
  1.0374193548387098,
  7.283870967741936],
 [95.42174024805198,
  0.5151599612158911,
  13.221703793258648,
  1.1486934169207217,
  1.7674652479484612]]

scaler_X = preprocessing.StandardScaler().fit([[0,0]])
scaler_T1 = preprocessing.StandardScaler().fit([[0]])
scaler_T2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3 = preprocessing.StandardScaler().fit([[0]])
scaler_T4 = preprocessing.StandardScaler().fit([[0]])

scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
scaler_T1.mean_, scaler_T1.scale_ = scaler_data[2][1], scaler_data[3][1] # solvent_aq
scaler_T2.mean_, scaler_T2.scale_ = scaler_data[2][2], scaler_data[3][2] # H3PO4_org
scaler_T3.mean_, scaler_T3.scale_ = scaler_data[2][3], scaler_data[3][3] # SO4_org
scaler_T4.mean_, scaler_T4.scale_ = scaler_data[2][4], scaler_data[3][4] # water_org


# scaler_T1 = preprocessing.StandardScaler().fit([[0]])
# scaler_T2 = preprocessing.StandardScaler().fit([[0]])
# scaler_T3 = preprocessing.StandardScaler().fit([[0]])
# scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

# scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
# scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
# scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# # scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
# scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
# scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [7]:
class EQUIL_rotem_extr1: #
# data8
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # scaler
        scaler_data=[[37.23870967741936, 7.199806451612903],
        [21.87534593155935, 6.51727589252697],
        [155.77096774193546,
          0.5283870967741936,
          13.459354838709675,
          1.0374193548387098,
          7.283870967741936],
        [95.42174024805198,
          0.5151599612158911,
          13.221703793258648,
          1.1486934169207217,
          1.7674652479484612]]

        scaler_X = preprocessing.StandardScaler().fit([[0,0]])
        scaler_T1 = preprocessing.StandardScaler().fit([[0]]) # solvent_aq
        scaler_T2 = preprocessing.StandardScaler().fit([[0]]) # H3PO4_org
        scaler_T3 = preprocessing.StandardScaler().fit([[0]]) # SO4_org
        scaler_T4 = preprocessing.StandardScaler().fit([[0]]) # water_org

        scaler_X.mean_, scaler_X.scale_ = scaler_data[0], scaler_data[1]
        scaler_T1.mean_, scaler_T1.scale_ = scaler_data[2][1], scaler_data[3][1] # solvent_aq
        scaler_T2.mean_, scaler_T2.scale_ = scaler_data[2][2], scaler_data[3][2] # H3PO4_org
        scaler_T3.mean_, scaler_T3.scale_ = scaler_data[2][3], scaler_data[3][3] # SO4_org
        scaler_T4.mean_, scaler_T4.scale_ = scaler_data[2][4], scaler_data[3][4] # water_org

        X_scaled=scaler_X.transform(x[:,[1,2]])
        X_scaled=np.clip(X_scaled, -2, 2)

        # H3PO4_org
        bp1=VectorBackProp(layers=[2,1,1], hidden_activation = 'sigmoid')
        manual_weights1=[[[-2.2542856480833247], [-0.7115892162097907]],
 [[2.11693061595761]],
 [[-4.9502436032281585]],
 [[3.912054301757645]]]
        bp1.import_weights(manual_weights1)
        predics1=scaler_T2.inverse_transform(bp1.run(X_scaled))

        # water_org
        bp2=VectorBackProp(layers=[2,2,1], hidden_activation = 'sigmoid')
        manual_weights2=[[[3.865511470369896, 8.917531695210526],
  [2.5196000386044317, 1.9258172086316168]],
 [[-2.2766428706488764, -1.5079705074058127]],
 [[2.7771600466732367], [0.9281118411212552]],
 [[-1.1710007634929602]]]
        bp2.import_weights(manual_weights2)
        predics2=scaler_T4.inverse_transform(bp2.run(X_scaled))

        # solvent_aq
        bp3=VectorBackProp(layers=[2,1,1], hidden_activation = 'linear')
        manual_weights3=[[[0], [0]], [[0]], [[0]], [[0.9]]]
        bp3.import_weights(manual_weights3)
        predics3=scaler_T1.inverse_transform(bp3.run(X_scaled))

        # SO4_org
        bp4=VectorBackProp(layers=[2,3,1], hidden_activation = 'sigmoid')
        manual_weights4=[[[5.255129630698163, 2.5370572590919305, -2.771603241282131],
  [4.30266945368854, 1.6643297587562575, 2.759812469181559]],
 [[-6.720893024720382, -0.7893443670545283, 5.835081175805133]],
 [[5.160798955335814], [2.555891510916236], [2.30456660012357]],
 [[-3.1529870476605937]]]
        bp4.import_weights(manual_weights4)
        predics4=scaler_T3.inverse_transform(bp4.run(X_scaled))

        self.xout = np.array([predics3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2[:,0], predics1[:,0], predics4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Cases

## Extraction 1

### test

In [8]:
x=np.array([[0,0,0],[100,100,100]])

In [9]:
X_scaled=scaler_X.transform(x[:,[1,2]])
X_scaled

array([[-1.70231409, -1.10472636],
       [ 2.86904219, 14.23910773]])

In [10]:
np.clip(X_scaled, -1, 1)

array([[-1., -1.],
       [ 1.,  1.]])

In [11]:
a=np.array([[0,30,9],[1,20,5]])
a[0]

array([ 0, 30,  9])

In [12]:
EQUIL_rotem_extr1(a).xout # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]

array([[ 0.99203106, 30.        ,  9.        ],
       [ 0.99203106, 20.        ,  5.        ]])

In [13]:
EQUIL_rotem_extr1(a).yout # [[%water_org, %H3PO4_org, %SO4_org], ...]

array([[5.51179427, 3.98631405, 0.76135935],
       [5.22454595, 0.7650427 , 0.16013486]])

In [14]:
test1 = LLC.Stage(173.2, 15, [8.2, .06, .06], [1, 30, 15], EQUIL_rotem_extr1, eff=.8, convergence=1E-4, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10,0],[5,5,0]])
test1.runs

10

In [83]:
test1.output()

Oout= 169.21927704579792
yout= [5.24911227 0.7879944  0.23470988]
yout_tag= [5.24911227 0.969993   0.27838735]  %water_org_equilib, %solubles_equilib
Aout= 15.323216002719382
xout= [ 0.99203106 16.93085744  9.58455678]
equilibrium efficiency= [1.  0.8 0.8]
error= [-1.70562754e-06 -6.25295869e-06  1.93580396e-04]
convergence= [ 8.91021042e-07  1.48212093e-05 -2.43776462e-05]


In [16]:
test2 = LLC.Stage(173.2, 50, [6, 6, 1], [1, 60, .1], EQUIL_rotem_extr1, eff=.8, convergence=1E-4, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10,1],[5,5,1]])
test2.runs

68

In [17]:
test2.output()

Oout= 187.0240741526847
yout= [ 7.18715462 11.47595873  0.68447158]
yout_tag= [ 7.18715462 12.84494841  0.60558947]  %water_org_equilib, %solubles_equilib
Aout= 32.01654194493727
xout= [ 0.99203106 50.51202203  1.3831771 ]
equilibrium efficiency= [1.  0.8 0.8]
error= [-5.31168917e-05 -4.00621298e-05 -1.86979252e-05]
convergence= [3.07111483e-05 2.35603290e-05 3.15455190e-05]


In [18]:
# test3 = LLC52.Stage(173.2, 50, [6, 6, .5], [1, 60, .1], EQUIL_rotem_extr1, eff=.8, convergence=1E-4)

### mesh

In [19]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 70, 31), np.linspace(0, 25, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh3 = x_mesh.flatten().reshape(31*31,1)
y_mesh3 = y_mesh.flatten().reshape(31*31,1)
mesh3=np.concatenate((x_mesh3, y_mesh3), axis=1)
# x_mesh[:5]

In [20]:
a=np.concatenate((np.zeros([len(mesh3),1]),mesh3), axis=1)

In [21]:
mesh_predict_x=np.array([EQUIL_rotem_extr1([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_extr1([row]).yout for row in a])
mesh_predics1_1=mesh_predict_y[:,:,1]
mesh_predics1_2=mesh_predict_y[:,:,0]
mesh_predics1_3=mesh_predict_x[:,:,0]
mesh_predics1_4=mesh_predict_y[:,:,2]

### 1 - Extraction1 - raffinate

In [22]:
# bat1_1=LLC.Battery(stages_num=3, Oin=173.2, Ain=15, yin=[8.2, .06, .06], xin=[1, 30, 15], EQUIL=EQUIL_rotem_extr1, eff=.8, max_iter=200)#, convergence=1E-4)
bat1_1=LLC.Battery(stages_num=3, Oin=173.2, Ain=50, yin=[6, 6, 1 ], xin=[1, 60, .1], EQUIL=EQUIL_rotem_extr1, method='secant', eff=.8, damping=.1, max_iter=30, battery_convergence=0.02, stage_convergence=1e-4, entrainment_perc_in=[1,2], entrainment_comp_in=[[10,10,10],[5,5,5]], entrainment_perc_out=[3,4])
bat1_1.runs

7

In [23]:
bat1_1.errors_array.plot()

In [24]:
bat1_1.outs_array.plot()

In [25]:
table1_1=LLC.BatteryTable(bat1_1).data

,Stage 1,Stage 2,Stage 3
"Org in, ton/hr",173.200000,175.404521,182.037106
"y1 in, %",6.000000,6.042117,6.756281
"y2 in, %",6.000000,7.076667,9.496326
"y3 in, %",1.000000,0.758882,0.643038
"Aq in, ton/hr",32.164299,38.886758,50.000000
"x1 in, %",0.992031,0.992031,1.000000
"x2 in, %",47.263654,52.698591,60.000000
"x3 in, %",1.830465,0.843602,0.100000
"Org out, ton/hr",175.366162,182.116231,192.414538
"y1 out, %",6.045201,6.765516,7.354981


In [26]:
# @markdown plots
fig = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

# visib='legendonly'
visib=True
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics1_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat1_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01), aspectratio=dict(x=1, y=1, z=1))
fig.update_scenes(camera_projection_type="orthographic")

fig.show()

## tests

i have matrix equation Ax=B. A is 2x2, x is 2x1, B is 2x1. I solve x=inv(A)B. I have another equation Cy=D. the same dimensions. how to combine solving x and y in a single equation?

how to code it if i have N such equations?


In [27]:
# Define the number of equations (N)
N = 3

# Define the matrices and vectors for each equation
# (Replace with your actual data)
matrices = [np.array([[1, 2], [3, 4]]),
            np.array([[5, 6], [7, 8]]),
            np.array([[9, 10], [11, 12]])]
vectors = [np.array([13, 14]),
           np.array([15, 16]),
           np.array([17, 18])]

# Create the block matrix and combined vector
block_matrix = np.zeros((N * 2, N * 2))  # Initialize with zeros
combined_vector = np.zeros(N * 2)

for i in range(N):
    block_matrix[i * 2: (i + 1) * 2, i * 2: (i + 1) * 2] = matrices[i]
    combined_vector[i * 2: (i + 1) * 2] = vectors[i]

# Solve for the combined solution vector
solution = np.linalg.solve(block_matrix, combined_vector)

# Extract individual solutions
solutions = [solution[i * 2: (i + 1) * 2] for i in range(N)]

# Print the solutions
for i, sol in enumerate(solutions):
    print(f"Solution for equation {i + 1}: x = {sol}")

Solution for equation 1: x = [-12.   12.5]
Solution for equation 2: x = [-12.   12.5]
Solution for equation 3: x = [-12.   12.5]


In [28]:
A,B=np.array([[5, 6], [7, 8]]),np.array([17, 18])
np.linalg.solve(A, B)

array([-14. ,  14.5])

In [29]:
Ain=bat1_1.Ain_list
Ain

array([32.16429949, 38.88675758, 50.        ])

In [30]:
stage=0
component=1
Ain,Oin=bat1_1.Ain_list[stage], bat1_1.Oin_list[stage]
xin=bat1_1.xin_list[stage,component]
yin=bat1_1.yin_list[stage,component]
xout=bat1_1.xout_list[stage,component]
yout=bat1_1.yout_list[stage,component]
yout

7.085237048371883

In [31]:
A,B=np.array([[1,1],[yout,xout]]), np.array([Oin+Ain,Oin*yin+Ain*xin])

In [32]:
np.linalg.solve(A, B)

array([173.65154956,  31.71274993])

In [33]:
# Define the number of equations (N)
bat=bat1_1
N = 3
component=1
# Define the matrices and vectors for each equation
matrices = []
vectors = []
for stage in range(N):
    matrix = np.array([[1,1],[bat.yout_list[stage,component],bat.xout_list[stage,component]]])
    matrices.append(matrix)
    vector = np.array([bat.Oin_list[stage]+bat.Ain_list[stage], bat.Oin_list[stage]*bat.yin_list[stage,component]+bat.Ain_list[stage]*bat.xin_list[stage,component]])
    vectors.append(vector)

# Create the block matrix and combined vector
block_matrix = np.zeros((N * 2, N * 2))  # Initialize with zeros
combined_vector = np.zeros(N * 2)

for i in range(N):
    block_matrix[i * 2: (i + 1) * 2, i * 2: (i + 1) * 2] = matrices[i]
    combined_vector[i * 2: (i + 1) * 2] = vectors[i]

# Solve for the combined solution vector
solution = np.linalg.solve(block_matrix, combined_vector)

solution.reshape(N,2)

array([[173.65154956,  31.71274993],
       [181.28686729,  33.00441089],
       [191.98641349,  40.05069281]])

In [34]:
Oout=solution.reshape(N,2)[:,0]
Oout[:-1]

array([173.65154956, 181.28686729])

In [35]:
table1_1

,Stage 1,Stage 2,Stage 3
"Org in, ton/hr",173.200000,175.404521,182.037106
"y1 in, %",6.000000,6.042117,6.756281
"y2 in, %",6.000000,7.076667,9.496326
"y3 in, %",1.000000,0.758882,0.643038
"Aq in, ton/hr",32.164299,38.886758,50.000000
"x1 in, %",0.992031,0.992031,1.000000
"x2 in, %",47.263654,52.698591,60.000000
"x3 in, %",1.830465,0.843602,0.100000
"Org out, ton/hr",175.366162,182.116231,192.414538
"y1 out, %",6.045201,6.765516,7.354981


## Extraction 2

In [36]:
class Klum:
  def __init__(self):
    pass

In [57]:
self=Klum()

In [66]:
self.stages_num=3
Oin=173.2
Ain=50
yin=[6, 6, 1 ]
xin=[1, 60, .1]
self.EQUIL=EQUIL_rotem_extr1

In [67]:
self.Oin_list = np.ones(self.stages_num) * Oin
self.Ain_list = np.ones(self.stages_num) * Ain
yin = np.array(yin, dtype=float) # [%water_org, %H3PO4_org, ...]
xin = np.array(xin, dtype=float) # [%solvent_aq, %H3PO4_aq, ...]
self.yin_list = np.array([yin for i in range(self.stages_num)])
self.xin_list = np.array([xin for i in range(self.stages_num)])
# self.Oout_list = np.zeros(self.stages_num)
# self.Aout_list = np.zeros(self.stages_num)
# self.yout_tag_list = np.zeros((self.stages_num, len(yin)))
# self.yout_list = np.zeros((self.stages_num, len(yin)))
# self.xout_list = np.zeros((self.stages_num, len(xin)))

In [69]:
self.yin_list

array([[6., 6., 1.],
       [6., 6., 1.],
       [6., 6., 1.]])

In [71]:
self.xout_list, self.yout_list = self.EQUIL(self.xin_list).xout, self.EQUIL(self.xin_list).yout

In [72]:
# Define the number of equations (N)
bat=self
N = self.stages_num
component=1
# Define the matrices and vectors for each equation
matrices = []
vectors = []
for stage in range(N):
    matrix = np.array([[1,1],[bat.yout_list[stage,component],bat.xout_list[stage,component]]])
    matrices.append(matrix)
    vector = np.array([bat.Oin_list[stage]+bat.Ain_list[stage], bat.Oin_list[stage]*bat.yin_list[stage,component]+bat.Ain_list[stage]*bat.xin_list[stage,component]])
    vectors.append(vector)

# Create the block matrix and combined vector
block_matrix = np.zeros((N * 2, N * 2))  # Initialize with zeros
combined_vector = np.zeros(N * 2)

for i in range(N):
    block_matrix[i * 2: (i + 1) * 2, i * 2: (i + 1) * 2] = matrices[i]
    combined_vector[i * 2: (i + 1) * 2] = vectors[i]

# Solve for the combined solution vector
solution = np.linalg.solve(block_matrix, combined_vector)

solution.reshape(N,2)

array([[257.87562671, -34.67562671],
       [257.87562671, -34.67562671],
       [257.87562671, -34.67562671]])

In [73]:
self.Oout_list, self.Aout_list = solution.reshape(N,2)[:,0], solution.reshape(N,2)[:,1]

In [74]:
# Substitution
self.Ain_list = np.concatenate([self.Aout_list[1:],[self.Ain_list[0]]])
self.Oin_list = np.concatenate([[self.Oin_list[0]],self.Oout_list[:-1]])
self.xin_list = np.concatenate([self.xout_list[1:],[self.xin_list[0]]])
self.yin_list = np.concatenate([[self.yin_list[0]],self.yout_list[:-1]])
# self.Oin_list
self.yin_list

array([[ 6.        ,  6.        ,  1.        ],
       [ 8.16965138, 23.73135329,  0.21361391],
       [ 8.16965138, 23.73135329,  0.21361391]])

In [103]:
def run(self):
    bat=self
    N = self.stages_num
    component=1
    # Define the matrices and vectors for each equation
    matrices = []
    vectors = []
    for stage in range(N):
        matrix = np.array([[1,1],[bat.yout_list[stage,component],bat.xout_list[stage,component]]])
        matrices.append(matrix)
        vector = np.array([bat.Oin_list[stage]+bat.Ain_list[stage], bat.Oin_list[stage]*bat.yin_list[stage,component]+bat.Ain_list[stage]*bat.xin_list[stage,component]])
        vectors.append(vector)

    # Create the block matrix and combined vector
    block_matrix = np.zeros((N * 2, N * 2))  # Initialize with zeros
    combined_vector = np.zeros(N * 2)

    for i in range(N):
        block_matrix[i * 2: (i + 1) * 2, i * 2: (i + 1) * 2] = matrices[i]
        combined_vector[i * 2: (i + 1) * 2] = vectors[i]

    # Solve for the combined solution vector
    solution = np.linalg.solve(block_matrix, combined_vector)

    return solution.reshape(N,2)

In [94]:
def display_table(batt, show=True, index_names=[]):
    len1=len(batt.xin_list[0])
    out=np.concatenate((
        [batt.Oin_list],
        batt.yin_list.T,
        [batt.Ain_list],
        batt.xin_list.T,
        [batt.Oout_list],
        batt.yout_list.T,
        [batt.Aout_list],
        batt.xout_list.T,
        ),axis=0)
    Name = (
        ['Org in, ton/hr']+
        ['y'+str(a+1)+' in, %' for a in range(len1)]+
        ['Aq in, ton/hr']+
        ['x'+str(a+1)+' in, %' for a in range(len1)]+
        ['Org out, ton/hr']+
        ['y'+str(a+1)+' out, %' for a in range(len1)]+
        ['Aq out, ton/hr']+
        ['x'+str(a+1)+' out, %' for a in range(len1)]
        )
    columns = ['Stage '+str(a) for a in range(1,batt.stages_num+1)]
    if len(index_names)>0:
        Name=index_names
    self.data=pd.DataFrame(out, columns=columns, index=Name)
    if show:
        display (self.data)

In [109]:
self.stages_num=3
Oin=173.2
Ain=50
yin=[6, 6, 1 ]
xin=[1, 60, .1]
self.EQUIL=EQUIL_rotem_extr1
self.Oin_list = np.ones(self.stages_num) * Oin
self.Ain_list = np.ones(self.stages_num) * Ain
yin = np.array(yin, dtype=float) # [%water_org, %H3PO4_org, ...]
xin = np.array(xin, dtype=float) # [%solvent_aq, %H3PO4_aq, ...]
self.yin_list = np.array([yin for i in range(self.stages_num)])
self.xin_list = np.array([xin for i in range(self.stages_num)])

In [112]:
self.xout_list, self.yout_list = self.EQUIL(self.xin_list).xout, self.EQUIL(self.xin_list).yout
out_flows = run(self)
self.Oout_list, self.Aout_list = out_flows.reshape(self.stages_num,2)[:,0], out_flows.reshape(self.stages_num,2)[:,1]
# Substitution
self.Oin_list[1:] = self.Oout_list[:-1]
self.yin_list[1:] = self.yout_list[:-1]
self.Ain_list[:-1] = self.Aout_list[1:]
self.xin_list[:-1] = self.xout_list[1:]
display_table(self)

,Stage 1,Stage 2,Stage 3
"Org in, ton/hr",173.200000,257.875627,257.875627
"y1 in, %",6.000000,8.169651,8.169651
"y2 in, %",6.000000,23.731353,23.731353
"y3 in, %",1.000000,0.213614,0.213614
"Aq in, ton/hr",50.000000,50.000000,50.000000
"x1 in, %",0.992031,0.992031,1.000000
"x2 in, %",60.000000,60.000000,60.000000
"x3 in, %",0.100000,0.100000,0.100000
"Org out, ton/hr",257.875627,257.875627,257.875627
"y1 out, %",8.169651,8.169651,8.169651


In [84]:
test1 = LLC.Stage(173.2, 50, [6, 6, 1], [1, 60, .1], EQUIL_rotem_extr1, eff=1, convergence=1E-4)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10,0],[5,5,0]])
test1.runs

83

In [85]:
test1.output()

Oout= 188.80075348566862
yout= [ 7.1364642  12.35799128  0.65244125]
yout_tag= [ 7.1364642  12.35799128  0.65244125]  %water_org_equilib, %solubles_equilib
Aout= 34.39924651433135
xout= [ 0.99203106 49.8086937   1.61556408]
equilibrium efficiency= [1. 1. 1.]
error= [-1.01144064e-05 -3.16286794e-03 -4.51031219e-03]
convergence= [ 2.12853660e-05 -7.12272172e-06 -9.50624859e-05]
